In [2]:
import torch
from torch.utils.data import Dataset
import numpy as np
from datasets import load_dataset
import re
import random
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import evaluate # ! pip install evaluate rogue_score

/Users/charanganesh/miniforge3/envs/AI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("LabHC/bias_in_bios", split='train[:20%]')
test_dataset = load_dataset("LabHC/bias_in_bios", split='test')
dev_dataset = load_dataset("LabHC/bias_in_bios", split='dev')

In [4]:
def neutralize_text(text: str) -> str:
    """Convert gendered pronouns to neutral forms"""
    replacements = [
        (r'\b([Hh]e|[Ss]he)\s+is\b', lambda m: "They are" if m.group(1)[0].isupper() else "they are"),
        (r'\b([Hh]e|[Ss]he)\s+was\b', lambda m: "They were" if m.group(1)[0].isupper() else "they were"),
        (r'\b([Hh]e|[Ss]he)\s+has\b', lambda m: "They have" if m.group(1)[0].isupper() else "they have"),
        (r'\b([Hh]e|[Ss]he)\b', lambda m: "They" if m.group(1)[0].isupper() else "they"),
        (r'\b([Hh]is|[Hh]er)\b', lambda m: "Their" if m.group(1)[0].isupper() else "their"),
        (r'\b([Hh]im|[Hh]er)\b', lambda m: "Them" if m.group(1)[0].isupper() else "them"),
        (r'\b([Hh]imself|[Hh]erself)\b', lambda m: "Themselves" if m.group(1)[0].isupper() else "themselves")
    ]
    for pattern, repl in replacements:
        text = re.sub(pattern, repl, text)
    return text

In [5]:
max_num_of_pairs: int = 5000
neutral_pairs = []
for i in range(min(max_num_of_pairs, len(dataset))):
    original = dataset[i]["hard_text"]
    neutral = neutralize_text(original)
    neutral_pairs.append((original, neutral))

In [6]:
random.shuffle(neutral_pairs)
train_size = int(0.8 * len(neutral_pairs))
train_data = neutral_pairs[:train_size]
val_data = neutral_pairs[train_size:]

In [7]:
MODEL_NAME = "t5-small"
TASK_PREFIX = "neutralize: "
MAX_LENGTH = 64
BATCH_SIZE = 32
EPOCHS = 2

In [8]:
class NeutralizationDataset(Dataset):
    def __init__(self, pairs, tokenizer):
        self.pairs = pairs
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        original, neutral = self.pairs[idx]
        inputs = self.tokenizer(
            TASK_PREFIX + original,
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        targets = self.tokenizer(
            neutral,
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": targets["input_ids"].squeeze(0).masked_fill(
                targets["input_ids"].squeeze(0) == self.tokenizer.pad_token_id, -100
            )
        }

In [9]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
train_dataset = NeutralizationDataset(train_data, tokenizer)
val_dataset = NeutralizationDataset(val_data, tokenizer)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="../models/neutralizer_model",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    fp16=torch.cuda.is_available(),
    logging_steps=100,
    save_total_limit=1,
    predict_with_generate=True,
    report_to="none" 
)

rouge = evaluate.load("rouge")

/Users/charanganesh/miniforge3/envs/AI/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

In [12]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    compute_metrics=compute_metrics
)

/var/folders/k1/pg7cnkbs3fzdgsq32r3ht2_80000gn/T/ipykernel_96284/1638589105.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [13]:
print("Training neutralization model...")
trainer.train()

Training neutralization model...


/Users/charanganesh/miniforge3/envs/AI/lib/python3.10/site-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.244600,0.164144,0.488161,0.472365,0.488311,0.488388
2,0.182700,0.159169,0.482502,0.466879,0.482700,0.482864


TrainOutput(global_step=250, training_loss=0.20589958572387695, metrics={'train_runtime': 186.6284, 'train_samples_per_second': 42.866, 'train_steps_per_second': 1.34, 'total_flos': 135341801472000.0, 'train_loss': 0.20589958572387695, 'epoch': 2.0})

In [14]:
model.save_pretrained("../models/neutralizer_model")
tokenizer.save_pretrained("../models/neutralizer_model")

('../models/neutralizer_model/tokenizer_config.json',
 '../models/neutralizer_model/special_tokens_map.json',
 '../models/neutralizer_model/spiece.model',
 '../models/neutralizer_model/added_tokens.json')

In [15]:
def neutralize_sentence(text):
    inputs = tokenizer(
        TASK_PREFIX + text,
        return_tensors="pt",
        max_length=MAX_LENGTH,
        truncation=True
    )
    outputs = model.generate(
        inputs.input_ids.to(model.device),
        attention_mask=inputs.attention_mask.to(model.device),
        max_length=MAX_LENGTH,
        num_beams=1  
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [24]:
test_cases = [
    "He is the project lead of EASy68K.",
    "She earned her degree from Michigan.",
    "The doctor told him that he needs to rest.",
    "The engineer told that he is busy, and she has to reschedule it"
]

In [25]:
print("\nNeutralization Examples:")
for case in test_cases:
    print(f"\nOriginal: {case}")
    print(f"Neutral: {neutralize_sentence(case)}")


Neutralization Examples:

Original: He is the project lead of EASy68K.
Neutral: They are the project lead of EASy68K.

Original: She earned her degree from Michigan.
Neutral: They earned their degree from Michigan.

Original: The doctor told him that he needs to rest.
Neutral: The doctor told them that they needs to rest.

Original: The engineer told that he is busy, and she has to reschedule it
Neutral: The engineer told that they are busy, and they have to reschedule it.
